In [ ]:
# focus only on common files
# fn to compare file name in common_files to section name
# if match, add a field to file called match_name and fill it with 1
# if not match, add a field to file called match_name and fill it with 0
# structure of comparison
# {"section_name" : {"match_name" : 1/0, "score" : 0.0}}
# score is the score of the file in the section
# compare the match_name field between two dicts
# if match_name is 1 then compare the score field
# if score is equal then add a field called model and fill it with the model name
# if score is not equal then add a field called model and fill it with the model name of the file with the highest score
# count each of model name (unique values)
# the one with higher count is more accurate

In [17]:
import json
import pandas as pd

In [18]:
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [19]:
def post_process(data):
    # remove the only_one_model key from each section since its empty anyway
    for section in data:
        del data[section]["only_one_model"]
    # convert each list in the common_files key to a key, value pair
    for key in data.keys():
        data[key] = {file[0]: file[1] for file in data[key]["common_files"]}
    return data

In [28]:
# for section in list of sections
# for file name in section
# remove suffix ".php" 
# if section name == file name
# add a field called retrieved and fill it with 1
# add a field called score and fill it with the score
# if section name != file name
# add a field called retrieved and fill it with 0
# add a field called score and fill it with the 0.0
def create_eval_dict(data):
    eval = {}
    for section in data:
        eval[section] = {}
        target_file = section + ".php"
        if target_file in data[section].keys():
            eval[section]["retrieved"] = 1
            eval[section]["score"] = data[section][target_file]
        else:
            eval[section]["retrieved"] = 0
            eval[section]["score"] = 0.0
    return eval


In [46]:
def convert_dict_df(data):
    # convert result dictionary into a dataframe with section names, files names, and scores as columns
    data_df = pd.DataFrame([
        {"section_name": section, "retrieved": metrics["retrieved"], "score": metrics["score"]}
        for section, metrics in data.items()
    ])
    return data_df

In [63]:
def get_retrieve_accuracy(eval_df):
    ones_count = int(eval_df[eval_df["retrieved"] == 1]["retrieved"].value_counts().iloc[0])
    print(f"Count of ones: {ones_count}")
    row_count = eval_df.shape[0]
    print(f"Count total observations: {row_count}")
    return ones_count / row_count * 100

In [68]:
azure_data = read_json_file("section_result_azure_embeddings.json")
azure_data = post_process(azure_data)
azure_eval = create_eval_dict(azure_data)
azure_eval_df = convert_dict_df(azure_eval)
azure_retrieve_accuracy = get_retrieve_accuracy(azure_eval_df)
azure_retrieve_accuracy

45
69


65.21739130434783

In [69]:
openai_data = read_json_file("section_result_openai_embeddings.json")
openai_data = post_process(openai_data)
openai_eval = create_eval_dict(openai_data)
openai_eval_df = convert_dict_df(openai_eval)
openai_retrieve_accuracy = get_retrieve_accuracy(openai_eval_df)
openai_retrieve_accuracy

42
69


60.86956521739131